## Importing dependencies

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import random
import pprint

## Getting proxies

In [ ]:
host = 'brd.superproxy.io'
port = 22225

username = '<username>'
password = '<password>'

proxy_url = f'http://{username}:{password}@{host}:{port}'

proxies = {
    'http': proxy_url,
    'https': proxy_url
}


url = "http://lumtest.com/myip.json"
response = requests.get(url, proxies=proxies)
pprint.pprint(response.json())

## Code to organize the scraped raw data

In [ ]:
def organize(raw):
  result = []
  for element in raw:
    a_tag = element.find_all('a')[0]
    href = a_tag.get('href')
    text = a_tag.get_text(strip=True)

    loc = element.find(class_ = "col-md-4")
    sales = element.find(class_ = "col-md-2 last").get_text().split('\n')[-1].strip()

    s = loc.get_text().split('\n')
    city, canton = s[5].strip().split(',')
    country = s[7].strip()

    result.append({'Company': text, 'a':href, 'city':city, 'canton':canton, 'country':country, 'sales':sales})

  return result


## Function for scraping the data from one page

In [ ]:
def get_details(page):

  cookies = {
      'site_preference': 'en-US',
      'SSLB': '1',
      'SSID': 'CQCSex0cAAAAAAD4urxmidfCGvi6vGYEAAAAAAAAAAAADZC9ZgB-5REwAQEVSiYADZC9ZgEAfSwBAYLyJQANkL1mAQA',
      'SSSC': '644.G7402997462253885321.4|76925.2486914:77841.2509333',
      'AKA_A2': 'A',
      'bm_ss': 'ab8e18ef4e',
      'SSRT': 'XpC9ZgADAA',
      'SSOD': 'AH7CAAAAEgAjzDMAGQAAAPi6vGZskL1mBQAAAA',
      'bm_s': 'YAAQxZUvF+4z3TWRAQAAHCx8VAGp/G9CA+j9WzljcT5wytTEetY8N3CGLnqUgPgnpmyIA6HUjJgjUxfl8QUVWhLk4oYlk83V654ngU201zQCzhj5lYsDz3omBuxugqvHeVG+jVzi0ufbpysIxtemlJ692VwnI6OJc7/iTOqOVb90uK9yE1it23htdAE2Q879qRQtxM78WGoYZEjN3MxacDaQQBr4UgpEH+iRmk9ZWoHj0baetIXkGUdvB23svoWH6ahkgI6IeL7lyLbrsX3t7oYp2YPpjcHmG0oe4252K/i2bmTCAzBQufMx58STdWkZXq3L5g9g6QCtM8lpo9McQp3JpaLSJZHK0HR2',
      'HID': '1723699311325',
      'cmapi_gtm_bl': 'ta-asp-bzi-sp-awct-cts-csm-img-flc-fls-mpm-mpr-m6d-tc-tdc',
      'cmapi_cookie_privacy': 'permit_1|2_functional',
      '_ga': 'GA1.1.2039139600.1723699311',
      '_mkto_trk': 'id:080-UQJ-704&token:_mch-dnb.com-1723699311400-79918',
      '_hjSessionUser_3331568': 'eyJpZCI6Ijk1YzRkYzExLTg2NmEtNTE1MS1hOGIxLTMwODYwODE0MmVkZiIsImNyZWF0ZWQiOjE3MjM2OTkzMTE0MTYsImV4aXN0aW5nIjpmYWxzZX0=',
      '_hjSession_3331568': 'eyJpZCI6ImQ2MTllOTgwLWMwMjgtNDJmMy1iZWFiLTIxMGI3NzJkZDk3YSIsImMiOjE3MjM2OTkzMTE0MTYsInMiOjAsInIiOjAsInNiIjowLCJzciI6MCwic2UiOjAsImZzIjoxLCJzcCI6MH0=',
      '_ga_K0F9RX4KM4': 'GS1.1.1723699311.1.0.1723699311.0.0.0',
      'bm_sz': '20BADE75CD50A36751295106ADD2BD11~YAAQxZUvF/Yz3TWRAQAAfi98VBjT81vqLddsBT+ft6RMe4SOMigJFweqCHg0uYQI4nHE13U7KpfTQ6kYLCgas/4NDNuhQWed+2+PyW+3h6uk8MRg4glnWr7M/yopC4R/CPwfUvd/Cnf/wEShcNUJD0sXyy9rJCRkBgOEf+JpYt3K9fORzFcXRGnbmWhsgBmMDRVpXggAfeBfDzZRGb1MTgDaBt++ZB2RtD8MHPLb57nc5L4hVA44ZJOpj0PRz7wnLjasJIBQxTXoYkx1RyOpVRR+42eImV0UcZ9eKNF60a5Nq37epH7RsWpiwjOprhWMiISl+DZsrH3hk5I/SXQmThIG94/d2R8w4G8HrRK887fI8xLA~3224633~4473412',
      '_abck': '51BBEABDB594DDE4EA8E0E7711A05C44~0~YAAQxZUvF/cz3TWRAQAAXjJ8VAzKi13dZIEqc5PrT3O13Q8M3OHh1CtAhJ35tTxSVcBNiVS1CQZmHsXvNUYalEvQI1xsTpwKnpgLzbFsFZvxtRJklG0z0wkGxkpWWbm7h+2sPxZIrJEU7WKtxVuU6c7MNLuF2ANOl/0m7rtDd6RUm3vQArTJatasFnAOz2VfxCmQHbnvkibT6X8FVfIvmvx3XRkcqBaoFW3Z0wA4+ul1oCpdQwnUGHMUktrYjejvkPCFt06bX5gQw905wQvqhGoSuziIzIGgv5HPu964zC+wKEYw5C9dqY4qBl3Ujod/xK3ceoKwA+akLLjLIqMUpidMJ1nPxVPkVkb/n3/WAawbAcbUkWep6C33Q0kg2WrG7mLpJyXwAOrXV6876EvqtFZxZ5N20lCSxB5IUrE+reIf/xBtzp3dHdmNW54YruBG+ojzOY3A1KBMbZlefim0J8jB~-1~||0||~-1',
      'ak_bmsc': '633A747474E0D726DE4C21E9775FB3E7~000000000000000000000000000000~YAAQxZUvF/gz3TWRAQAA8TJ8VBjmpUaJ+KTX1arbHziCFiRIhJAIPG39UHDd0o3ZvFmsmaqpg8M0ObNGG3kJHJSQFKKt3kx9+903BUFIYPzg3e0ytS8wc89fbZAvWH5uugibwCV5N9qO/eOFQfeqvpycHXJhSzMzWErfhdf/BL6hb8Kq+jU4gjhUJtqgf1WH/e0r+sShu1v1anaQoFfE/wDbZhi3pV6cBMd9seOKdMr+6+RMBHPs0SS/3gTLxJyXI2/VHgWVrwr6uHe9dDn3qdlEFrgn9oUwiYLvANs+nAwvopGKWwOtJTUBbsgnpgJIsffDCcmk3h4u+qBKRlKXuuRDbN5nBLsmqWRMs+JHLlU9t5UkpVJ5tG5KNaFMX4oKIMtjBBb87w==',
  }

  user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/127.0.0.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0'
    ]

  accept_languages = [
    'en-US,en;q=0.9',
    'en-GB,en;q=0.8',
    'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'de-DE,de;q=0.9,en-US;q=0.8,en;q=0.7',
    'es-ES,es;q=0.9,en-US;q=0.8,en;q=0.7',
    'zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'ja-JP,ja;q=0.9,en-US;q=0.8,en;q=0.7'
    ]

  x = random.randint(11,30)

  accepts = [
    'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    ]

  headers = {
      'accept': random.choice(accepts),
      'accept-language': random.choice(accept_languages),
      'DNT': '1',
      'Connection': 'keep-alive',
      'Cache-control': 'max-age=0',
      'priority': 'u=0, i',
      'referer': f'https://www.dnb.com/business-directory/company-information.manufacturing.ch.html?page={x}',
      #'sec-ch-ua': '"Not)A;Brand";v="99", "Microsoft Edge";v="127", "Chromium";v="127"',
      'Sec-Fetch-Dest': 'document',
      'Sec-Fetch-Mode': 'navigate',
      'Sec-Fetch-Site': 'none',
      'Sec-Fetch-User': '?1',
      'Sec-Ch-Ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
      'Sec-Ch-Ua-Mobile': '?0',
      'Sec-Ch-Ua-Platform': '"Windows"',
      'Upgrade-insecure-requests': '1',
      'User-agent': random.choice(user_agents),
  }

  params = {
      'page': f'{page}',
  }

  response = requests.get(
      'https://www.dnb.com/business-directory/company-information.manufacturing.ch.html',
      params=params,
      cookies=cookies,
      headers=headers,
      proxies=proxies
  )

  soup = BeautifulSoup(response.text, 'html.parser')
  raw = soup.find_all(class_="col-md-12 data")

  return organize(raw)

## Main code that assembles everything together.


In [ ]:
data = pd.DataFrame()
for page in tqdm(range(16, 21)):
  new = get_details(page)
  new = pd.DataFrame(new)
  data = pd.concat([data, new], ignore_index = True)
  time.sleep(random.randint(1,10))